# Qcodes example with Swabian Instruments Time Tagger

In [ ]:
from qcodes_contrib_drivers.drivers.SwabianInstruments.Swabian_Instruments_Time_Tagger import TimeTagger

The driver wraps the `TimeTagger` python API provided by Swabian Instruments (download [here](https://www.swabianinstruments.com/time-tagger/downloads/)). As the API is very object-oriented, the QCoDeS driver is fairly dynamic in that, for each new measurement type, a new `InstrumentChannel` of the corresponding type is added to the main instrument. These channels have parameters which correspond to the instantiation arguments of the API objects, as well as gettable parameters which correspond to the measured data.

For more information on the driver design, see the `SwabianInstruments/Swabian_Instruments_Time_Tagger.py` module docstring.

In [ ]:
tagger = TimeTagger('tagger')

To add a new `Correlation` measurement, use the automatically generated helper method `add_correlation_measurement`, which adds a `CorrelationMeasurement` channel to the `correlation_measurements` channel list.

In [ ]:
correlation = tagger.add_correlation_measurement()
correlation

At first, the API object is not yet instantiated because arguments are missing:

In [ ]:
try:
    correlation.api
except RuntimeError as err:
    print(err)

Instead, we must provide the arguments by setting the parameters:

In [ ]:
correlation.channels([1, 2])
correlation.binwidth(100)
correlation.n_bins(1000)
correlation.api

We can now start measuring:

In [ ]:
correlation.start_for(5, clear=True)
while correlation.is_running():
    pass
print(correlation.capture_duration())
print(correlation.data_normalized())

Once a measurement is not needed anymore, it may also be removed from the tagger object:

In [ ]:
tagger.correlation_measurements.remove(correlation)

## Synchronized measurements

The `TimeTagger` driver also implements the `SynchronizedMeasurements` functionality of the API. This is an object that helps to synchronize multiple different measurements using the same physical tagger. It can either be used by passing the virtual tagger return by its `get_tagger()` method to the measurement, or by calling the `register_measurement()` and `unregister_measurement()` methods:

In [ ]:
tagger.synchronized_measurements

In [ ]:
sync_tagger = tagger.synchronized_measurements.get_tagger()
sync_correlation = tagger.add_correlation_measurement(api_tagger=sync_tagger)
sync_count_rate = tagger.add_count_rate_measurement(api_tagger=sync_tagger)
# Perform measurement as usual

Or, equivalently:

In [ ]:
correlation = tagger.add_correlation_measurement()
count_rate = tagger.add_count_rate_measurement()
tagger.synchronized_measurements.register_measurement(correlation)
tagger.synchronized_measurements.register_measurement(count_rate)
# Perform measurement as usual
tagger.synchronized_measurements.unregister_measurement(correlation)
tagger.synchronized_measurements.unregister_measurement(count_rate)